In [17]:
#from qiskit import *
from qiskit.quantum_info import SparsePauliOp, Statevector
# from VQE_CVaR import partition_N
import numpy as np
from qiskit import QuantumRegister, QuantumCircuit

from math import acos
from scipy.optimize import minimize


#from qubo_hamiltonian import *
from Functions import *

import networkx as nx
from networkx import random_graphs

import matplotlib.pyplot as plt

Try different orderings

In [18]:
def sorted_gates(coefficients, edges, Abs, invert):

    if Abs:
        coefficients = np.abs(coefficients)

    sorted_index = np.argsort(coefficients)

    if invert:
        sorted_index = sorted_index[::-1]

    sorted_edges = [edges[x] for x in sorted_index]

    grouped = []
    group = []

    for t in sorted_edges:
        qi, qj = t
        if any(qi in pair for pair in group) or any(qj in pair for pair in group):
            grouped.append(group)
            group = []
        group.append(t)  
    grouped.append(group)

    if len(list(itertools.chain.from_iterable(grouped))) != len(edges):
        sys.stderr.write('something is wrong with the new columns')
        sys.exit()

    return sorted_edges, grouped 


def sorted_columns(edge_coeff_dict, edges_columns, Abs, invert):
    sum_jcol = []

    if Abs:
        for column in edges_columns:
            abs_sum_j = 0
            for edge in column:
                abs_sum_j += abs(edge_coeff_dict[edge])
            sum_jcol.append(abs_sum_j)

    else:
        for column in edges_columns:
            sum_j = 0 
            for edge in column:
                sum_j += edge_coeff_dict[edge]
            sum_jcol.append(sum_j)

    new_index = np.argsort(sum_jcol)
    
    if invert:
        new_index = new_index[::-1]

    sorted_edge_columns = copy.deepcopy(np.array(edges_columns,  dtype=object)[new_index]) 
    return sorted_edge_columns.tolist()


In [19]:
def adjoint_matrix(N, edge_coeff_dict):
    adjmat = np.zeros((N, N))
    for edge, coeff in edge_coeff_dict.items():
        if len(edge) == 2:
            i, j = edge
            adjmat[i, j] = coeff/2
            adjmat[j, i] = coeff/2
        else:
            i = edge[0]
            adjmat[i, i] = coeff
    return adjmat

def initial_state_ry(N:int, z_list):
    """initialize the quantum state of each qubit n as: ry(a)|0> = (cos(a/2)|0> + sin(a/2) |1>), <Z> = cos(a/2)^2 - sin(a/2)^2 = cos(a)
    Args:
        N: number of qubits
        z_list: list of expectation values of sigma^z for each qubit
    Returns:
        circ_init: QuantumCircuit for the initial state
    
    """
    q = QuantumRegister(N)
    circ_init = QuantumCircuit(q)

    for n in range(N):
        # find the angle theta such that sin(theta/2)**2 * C = k_list[n]
        expz = z_list[n]
        theta = acos(expz)
        circ_init.ry(theta, q[n])

    return circ_init

def get_expz(N, state_vector, alpha, eigen_list):
    """get the expectation value of <Z> for each qubit
    Args:
        N: number of qubits
        state_vector: state vector of the quantum circuit
        alpha: CVaR coefficient
    Returns:
        z_list: list of expectation values of sigma^z for each qubit
    """
    # backend = Aer.get_backend('statevector_simulator')
    # job = backend.run(circ)
    # result = job.result()
    # outputstate = np.array(result.get_statevector(circ))   ### amplitude
    # prob_list = [abs(outputstate[i]) ** 2 for i in range(len(outputstate))]
    prob_list = np.abs(state_vector) ** 2


    sorted_indices = np.argsort(eigen_list)
    probs = np.array(prob_list)[sorted_indices]
    vals = np.array(eigen_list)[sorted_indices]

    probs_1 = []  ### pribality of the eigenvalues counted in CVaR
    vals_1 = []   ### eigenvalues counted in CVaR

    cvar = 0
    total_prob = 0
    expz_array = np.zeros(N)
    for i, (p, v) in enumerate(zip(probs, vals)):
        if p >= alpha - total_prob:
            p = alpha - total_prob
        total_prob += p
        cvar += p * v

        bits = format(sorted_indices[i], '0' + str(N) + 'b')[::-1] # bitstring of the eigen state
        for j, bit in enumerate(bits):
            expz_array[j] += pow(-1, int(bit)) * p

        if abs(total_prob - alpha) < 1e-8:
            break

    cvar /= total_prob
    expz_array /= total_prob
   
    return probs[0], cvar, expz_array

In [23]:
#SORTING THE GATES

# Set the seed
np.random.seed(52)

N = 8
Numpara = 4
ntrials = 10

#Def graph with edges
G = nx.complete_graph(N)
pairs_all = list(itertools.chain.from_iterable(partition_N(N)))
edges_columns = partition_N(N)
lightcone_dict = find_light_cone(edges_columns)


ITE_poss = []

for trial in range(10): 
    print('\n\n#############################################################################')
    print(f"Trial: {trial}")

    #Set random coefficients and correspoding Hamiltonian
    coeff_list = np.random.uniform(-1, 1, N + len(G.edges()) )
    h_list = coeff_list[: N ]
    J_list = coeff_list[N :]
    H = Hamiltonian_qubo(N, pairs_all, h_list, J_list)
    eigen_list = H.to_spmatrix().diagonal().real

    eigens_ids = np.argsort(eigen_list)[:3]
    for id in eigens_ids:
        print('eigenvalue', eigen_list[id], ',  bitstring', format(id, f'0{N}b')[::-1])

    #Make edge coeff dictionary
    edge_coeff_dict = {}
    edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(h_list)})
    edge_coeff_dict.update({edge: J_val for edge, J_val in zip(pairs_all, J_list)})
    print('edge_coeff_dict', edge_coeff_dict)

    if trial in [8]:
        for tau in [0.1, 0.2]:
            print("\n############################################ tau: ", tau)
            print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ITE')
            exp_poss_dict, state_ite = ITE(N, edge_coeff_dict, tau, eigen_list)
            ITE_poss.append(list(exp_poss_dict.items())[0][1])
            print('exp_poss_dict', list(exp_poss_dict.items())[0])

            
            Ampth = 0.0 
            for alpha in [0.1]:
                print(f'\n## alpha: {alpha}')

                print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Warm start')
                expz_array = np.array([0]*N)
                for step in range(3):
                    print(f'# step: {step}')
                    circ_init = initial_state_ry(N, expz_array)
                    edge_params_dict, params_init, exp_poss_dict, state_all = warm_start_parameters_adaptlightcone(N, tau, Numpara, circ_init, 0.0, edge_coeff_dict, edges_columns, eigen_list, 1, False)
                    poss, cvar, expz_array = get_expz(N, state_all, alpha, eigen_list)
                    print('exp_poss_dict', list(exp_poss_dict.items())[:10])
                    print("fidelity: ", poss, ",  cvar: ", cvar, ",  expz_array: ", expz_array)


                for Abs in [True, False]:
                    for invert in [True, False]:
                        print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ SORTED {Abs} {invert}')
                        #Sort gates according to J
                        sorted_edges, sorted_edge_columns = sorted_gates(coeff_list[N :], pairs_all, Abs, invert)
                        sorted_lightcone_dict = find_light_cone(sorted_edge_columns)

                        expz_array = np.array([0]*N)
                        for step in range(3):
                            circ_init = initial_state_ry(N, expz_array)
                            edge_params_dict, params_init, exp_poss_dict, state_all = warm_start_parameters_adaptlightcone(N, tau, Numpara, circ_init, 0.0, edge_coeff_dict, sorted_edge_columns, eigen_list, 1, False)
                            poss, cvar, expz_array = get_expz(N, state_all, alpha, eigen_list)
                            print('exp_poss_dict', list(exp_poss_dict.items())[:10])
                            print("fidelity: ", poss, ",  cvar: ", cvar, ",  expz_array: ", expz_array)



#############################################################################
Trial: 0
eigenvalue -16.76217612238438 ,  bitstring 1001010010
eigenvalue -14.709210245769828 ,  bitstring 1001010110
eigenvalue -13.464499343004558 ,  bitstring 1001010000
edge_coeff_dict {(0,): 0.6462206814195839, (1,): -0.9477640368602653, (2,): -0.5784587201374121, (3,): 0.23684353869922048, (4,): -0.8034310693262017, (5,): 0.24026261975371765, (6,): -0.8922195608031125, (7,): 0.921308115608477, (8,): 0.960858748430147, (9,): 0.04225530054244775, (0, 1): 0.27310668967109564, (2, 3): 0.5295138965384998, (4, 5): 0.5299105892336384, (6, 7): -0.16462884088055452, (8, 9): 0.5376106126474853, (0, 3): -0.15359649917593665, (2, 5): 0.8522071430536631, (4, 7): 0.36385296974479675, (6, 9): -0.26308881735062317, (1, 3): 0.7178197107056401, (0, 5): -0.2390086400332203, (2, 7): -0.8100914722327845, (4, 9): -0.3502185727263536, (6, 8): -0.16977562771501753, (1, 5): 0.48454789770076045, (0, 7): 0.3158177351732514, (2,

KeyboardInterrupt: 